In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
pd.options.display.float_format = '{:,.0f}'.format

In [4]:
train_reader = pd.read_csv('Data/Springleaf/train.csv.zip', sep=',', iterator=True, low_memory=False)
test_reader = pd.read_csv('Data/Springleaf/test.csv.zip', sep=',', iterator=True, low_memory=False)

In [5]:
train = train_reader.get_chunk(30000)

In [7]:
test = test_reader.get_chunk(10000)

In [76]:
df_train = train.copy()

In [9]:
train.shape, test.shape

((30000, 1934), (10000, 1933))

In [10]:
def display_all(df):
    with pd.option_context('display.max_rows', 10000, 'display.max_columns', 10000):
        display(df)

In [20]:
display_all(df_train.head(10))

ID VAR_0001  VAR_0002  VAR_0003  VAR_0004 VAR_0005  VAR_0006  VAR_0007  \
0   2        H       224         0      4300        C         0         0   
1   4        H         7        53      4448        B         1         0   
2   5        H       116         3      3464        C         0         0   
3   7        H       240       300      3200        C         0         0   
4   8        R        72       261      2000        N         0         0   
5  14        R         4         4      4422        C         0         0   
6  16        H        60       132     40000        C         1         1   
7  20        R        13        75      3600        B         0         0   
8  21        R        17        16      2296        N         1         1   
9  22        R        24        72       450        N         0         0   

  VAR_0008 VAR_0009 VAR_0010 VAR_0011 VAR_0012  VAR_0013  VAR_0014  VAR_0015  \
0    False    False    False    False    False         0         0         0   
1    False    False    False    False    False         1         0         1   
2    False    False    False    False    False         0         0         0   
3    False    False    False    False    False         0         0         0   
4    False    False    False    False    False         0         0         0   
5    False    False    False    False    False         0         0         0   
6    False    False    False    False    False         1         1         1   
7    False    False    False    False    False         0         0         0   
8    False    False    False    False    False         1         1         1   
9    False    False    False    False    False         0         0         0   

   VAR_0016  VAR_0017  VAR_0018  VAR_0019  VAR_0020  VAR_0021  VAR_0022  \
0         1         0         0         0         0         0         0   
1         2         1         0         0         0         0         0   
2         1         0         0         0         0         0         0   
3         2         0         0         0         0         0         0   
4         1         0         0         0         0         0         0   
5         1         0         0         0         0         0         0   
6         2         0         0         0         0         0         0   
7         1         0         0         0         0         0         0   
8         1         1         0         0         0         0         0   
9         1         0         0         0         0         0         0   

   VAR_0023  VAR_0024  VAR_0025  VAR_0026  VAR_0027  VAR_0028  VAR_0029  \
0         0         0         0         0         0         0         0   
1         0         0         0         0         0         0         0   
2         0         0         0         0         0         0         0   
3         0         0         0         0         0         0         0   
4         0         0         0         0         0         0         0   
5         0         0         0         0         0         0         0   
6         0         0         0         0         0         0         0   
7         0         0         0         0         0         0         0   
8         0         0         0         0         0         0         0   
9         0         0         0         0         0         0         0   

   VAR_0030  VAR_0031  VAR_0032  VAR_0033  VAR_0034  VAR_0035  VAR_0036  \
0         0         0         0         1         0         0         0   
1         0         0         0         1         0         0         0   
2         0         0         0         1         0         0         0   
3         0         0         0         1         0         0         0   
4         0         0         0         1         0         0         0   
5         0         0         0         1         0         0         0   
6         0         0         0         2         2         2         1   
7         0         0 

Before doing any EDA, by eyeballing the data I can see that so many columns are repeated

In [91]:
# Drop VAR_0044 after noticing that it only holds empty lists
df_train.drop('VAR_0044', axis = 1, inplace=True)

In [170]:
# Extract numerical variables to run diff function
from pandas.api.types import is_string_dtype, is_numeric_dtype

def numeric(df):
    lists=[]
    for name, col in df.items():
        if is_numeric_dtype(df[name]):
            lists.append(name)
    return df[lists]

In [225]:
df_train.shape

(30000, 1933)

In [226]:
# Drop columns with total value of 0
for name, col in df_train.items():
    if is_numeric_dtype(df_train[name]) and df_train[name].sum() == 0:
        df_train.drop(name, axis =1, inplace=True)

In [227]:
df_train.shape

(30000, 1894)

Dropped 39 columns that had no values

In [229]:
df_num = numeric(df_train)
df_diff = df_num.diff(axis =1)

In [234]:
# Drop columns with total difference value of 0 which will eliminate variables with no values as they are replica of preceeding column
for name, col in df_train.items():
    if is_numeric_dtype(df_train[name]) and df_diff[name].sum() == 0:
        df_train.drop(name, axis =1, inplace=True)

In [235]:
df_train.shape

(30000, 1886)

Dropped 8 columns that are replica of the preceeding and thus provides no value

In [230]:
display_all(df_diff.sum())

ID                              0
VAR_0002             -890,752,922
VAR_0003                 -483,782
VAR_0004              106,218,330
VAR_0006                        0
VAR_0007                   -5,239
VAR_0013                    5,239
VAR_0014                   -5,760
VAR_0015                     -618
VAR_0016                   29,033
VAR_0017                  -28,463
VAR_0033                   18,350
VAR_0034                  -19,446
VAR_0035                   -2,255
VAR_0036                   -1,213
VAR_0037                   -1,273
VAR_0045                   -8,785
VAR_0046                      302
VAR_0047                      338
VAR_0048                      327
VAR_0049                    3,308
VAR_0050                    5,162
VAR_0051                      937
VAR_0052                   21,215
VAR_0053                    1,873
VAR_0054                    1,626
VAR_0055                    1,156
VAR_0056                    5,741
VAR_0057                    2,121
VAR_0058      

In [78]:
# Convert these to datetime
dates = ['VAR_0178', 'VAR_0179', 'VAR_0217', 'VAR_0073', 'VAR_0075', 'VAR_0166', 'VAR_0168', 'VAR_0169', 'VAR_0176']

for d in dates:
    df_train[d] = df_train[d].str[:7]
    df_train[d] = pd.to_datetime(df_train[d], dayfirst=True)

In [ ]:
# Convert objects into categories

In [ ]:
# What is it with the variables -1

In [25]:
# Drop Var_0202 as it holds no value
df_train.VAR_0202.value_counts()

BatchInquiry    29991
Name: VAR_0202, dtype: int64

In [12]:
df_train.target.value_counts()

0    23098
1     6902
Name: target, dtype: int64

Check if there are any fully empty variables

In [13]:
df_train.isnull().sum().sort_values(ascending=False).head(20)

VAR_0207    30000
VAR_0213    30000
VAR_0840    30000
VAR_0214    29999
VAR_0157    29801
VAR_0158    29573
VAR_0205    29566
VAR_0206    29551
VAR_0167    29505
VAR_0177    29334
VAR_0156    28784
VAR_0159    28784
VAR_0209    28080
VAR_0168    27750
VAR_0178    27477
VAR_0169    27035
VAR_0166    27035
VAR_0179    26350
VAR_0176    26350
VAR_0208    25995
dtype: int64

In [14]:
# Drop variables that contain no data
df_train.drop(['VAR_0207', 'VAR_0213', 'VAR_0840', 'VAR_0207'], axis=1, inplace=True)

In [22]:
# This looks like a date 



NameError: name 'VAR_0073' is not defined